<a href="https://colab.research.google.com/github/seungheee/algoStudy/blob/master/%EC%8B%A4%EC%8A%B5_BERT_%ED%86%A0%ED%81%AC%EB%82%98%EC%9D%B4%EC%A0%80_%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BERT 토크나이저 실습

이번 실습에서는 구글에서 공개한 Multi-lingual BERT를 다운로드해 사용해보겠습니다.   



## #1. 필요한 라이브러리 설치 및 로딩

#### ▶ pip로 bert-for-tf2 설치하기

bert-for-tf2 패키지를 사용하면 BERT tokenizer을 아주 쉽게 사용할 수 있습니다.  




In [2]:
!pip install bert-for-tf2
!pip install konlpy

#### ▶ 필요한 라이브러리 로딩
방금 설치한 bert 패키지와 TensorFlow Hub를 로딩하겠습니다.

In [3]:
## bert 모듈 로딩 & TF hub 연결

import bert
import tensorflow_hub as hub

## #2. 사전학습된 BERT 모델 로딩
Tensorflow hub에서 pretrain된 다국어 BERT 모델을 가져오는 코드입니다.   
홈페이지에서 Multi-lingula BERT에 해당하는 주소를 복사해 BERT_MODEL_HUB에 입력했습니다. 

그리고 hub.KerasLayer 함수를 이용해 bert_layer를 가지고 왔습니다.   
이 레이어가 바로 Transformer 인코더가 12층 쌓여있는 BERT 모델입니다.   




In [4]:
BERT_MODEL_HUB = 'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/2'

# BERT layer 가져오기
bert_layer = hub.KerasLayer(BERT_MODEL_HUB, trainable=True)

## #3. BERT parsing 이해하기
- BERT에서는 Wordpiece Tokenization을 통해 토큰을 subtoken으로 쪼갭니다.    
- 한국어의 경우 원형을 보존하는 형태소 분석을 거친 후 subtoken으로 쪼개는 것이 좋습니다.   
- 위에서 로딩한 bert_layer에서 사전학습에 활용한 토크나이저를 로딩할 수 있습니다.  
   - bert.tokenization.bert_tokenization 함수 사용
   - 형태소 분석기를 이용해 문장을 형태소 단위로 쪼갠 후
   - tokenizer의 <font color="blue">FullTokenizer</font> 을 사용해 Sub-tokenization 진행

- 우리가 Python으로 코딩했던 @convert_tokens_to_ids나 @convert_ids_to_tokens매서드가 bert 패키지에 모두 포함되어 있습니다!

#### Step 1. 토크나이저 로딩하기

In [5]:
from  bert.tokenization import bert_tokenization

# vocab_file 가져오기
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()

# 소문자화를 하는지 여부 가져오기
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

# 토크나이저 로딩
print("vocab file:", vocab_file)
print("do_lower_case:", do_lower_case)

tokenizer = bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

vocab file: b'/tmp/tfhub_modules/3e9209b9f2a53dfa4e6d93250dfceb5e64d73b66/assets/vocab.txt'
do_lower_case: False


👉 BERT에서 사용하는 단어사전이 위에 프린트된 경로에 txt 파일로 저장되어 있습니다.   
👉 소문자화를 진행하여 학습한 BERT도 있지만 다국어 모델은 소문자화를 하지 않았기 때문에 do_lower_case=False인 것을 볼 수 있습니다.   

In [6]:
# vocab 사전 확인하기

print("단어사전에 있는 토큰 개수:", len(tokenizer.vocab))
print("예시:", list(tokenizer.vocab.keys())[:300])

단어사전에 있는 토큰 개수: 119547
예시: ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused32]', '[unused33]', '[unused34]', '[unused35]', '[unused36]', '[unused37]', '[unused38]', '[unused39]', '[unused40]', '[unused41]', '[unused42]', '[unused43]', '[unused44]', '[unused45]', '[unused46]', '[unused47]', '[unused48]', '[unused49]', '[unused50]', '[unused51]', '[unused52]', '[unused53]', '[unused54]', '[unused55]', '[unused56]', '[unused57]', '[unused58]', '[unused59]', '[unused60]', '[unused61]', '[unused62]', '[unused63]', '[unused64]', '[unused65]', '[unused66]', '[unused67]', '[unused68]', '[unused69]', '[unus

👉 총 119,547개의 토큰이 포함되어 있습니다.   
👉 [PAD] 토큰부터 시작해 unused로 예약된 자리가 있고, 영어, 러시아어(?) 등 다양한 언어의 토큰들이 포함되어 있습니다.   
👉 한국어만을 위한 모델이 아니기 때문에, 한국어만으로 사전학습한 BERT에 비해서는 성능이 떨어집니다 ㅠ.ㅠ

In [7]:
""" 형태소 분석 함수 """
from konlpy.tag import Okt
okt=Okt()

def tokenize(lines):
  return okt.morphs(lines)

👉 1~2일차 실습에서 저희는 Komoran 형태소분석기를 사용했습니다.    
하지만 Komoran은 원형을 복원하여 형태소를 분석하는 아이였습니다.   

👉 Sub-tokenizing을 위해서는 문장을 그대로 쪼개기만 해야 하기 때문에, okt 분석기를 사용하였습니다. 

#### Step 2. 형태소 분석 + Subtokenization 실행하기
- tokenizer.tokenize 사용

In [8]:
sentence = "버트로 토크나이즈하는 예제"

# basic_tokenizer로 문장 쪼개기
tokenized_sentence = tokenize(sentence)
print(tokenized_sentence)

# BPE로 문장 쪼개기
sub_tokens = tokenizer.tokenize(" ".join(tokenized_sentence))
print(sub_tokens)

['버트', '로', '토크', '나', '이즈', '하', '는', '예제']
['버', '##트', '로', '토', '##크', '나', '이', '##즈', '하', '는', '예', '##제']


👉 tokenizer.tokenize 함수를 통해 형태소 분석된 문장을 WordPiece 단위로 쪼갭니다. 

🙆‍♀️ 원하는 자연어 문장을 BERT tokenizer로 쪼개고 결과를 확인해 보세요

In [9]:
sentence = "햄버거는 역시 버거킹"

# basic_tokenizer로 문장 쪼개기
tokenized_sentence = tokenize(sentence)
print(tokenized_sentence)

# Sub-token으로 쪼개기
print(tokenizer.tokenize(" ".join(tokenized_sentence)))

['햄버거', '는', '역시', '버거킹']
['햄', '##버', '##거', '는', '역시', '버', '##거', '##킹']


#### Step 3. BPE 토큰을 모델 인풋 인덱스로 바꾸기
- tokenizer.convert_tokens_to_ids를 사용하면 Subtoken을 인덱스로 바꿀 수 있습니다. 
- 우리가 코딩해서 사용했던 방식과 동일하게 작동합니다. 

In [10]:
# 모델 인풋 인덱스로 바꾸기
print(sub_tokens)
input_ids = tokenizer.convert_tokens_to_ids(sub_tokens)
print(input_ids)

['버', '##트', '로', '토', '##크', '나', '이', '##즈', '하', '는', '예', '##제']
[9336, 15184, 9202, 9873, 20308, 8982, 9638, 24891, 9952, 9043, 9576, 17730]


In [11]:
# 인풋 인덱스를 토큰으로 바꾸기
reversed_token = tokenizer.convert_ids_to_tokens(input_ids)
print(reversed_token)

['버', '##트', '로', '토', '##크', '나', '이', '##즈', '하', '는', '예', '##제']


## #4. BERT vocab 커스터마이즈하기
BERT에는 무려 99개의 unused 토큰 자리가 예약되어 있습니다.   
이 자리를 어떤 식으로 활용할 수 있을까요?

먼저 원래 단어사전 text 파일을 열어 org_vocabs라는 리스트에 읽어오겠습니다. 

In [12]:
## 원래 단어 사전 확인하기

with open(vocab_file) as f:
  org_vocabs = [s.strip() for s in f.readlines()]

In [13]:
print("# vocabs:", len(org_vocabs))
print(org_vocabs[:101])

# vocabs: 119547
['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused32]', '[unused33]', '[unused34]', '[unused35]', '[unused36]', '[unused37]', '[unused38]', '[unused39]', '[unused40]', '[unused41]', '[unused42]', '[unused43]', '[unused44]', '[unused45]', '[unused46]', '[unused47]', '[unused48]', '[unused49]', '[unused50]', '[unused51]', '[unused52]', '[unused53]', '[unused54]', '[unused55]', '[unused56]', '[unused57]', '[unused58]', '[unused59]', '[unused60]', '[unused61]', '[unused62]', '[unused63]', '[unused64]', '[unused65]', '[unused66]', '[unused67]', '[unused68]', '[unused69]', '[unused70]', '[

이번 프로젝트로 LG CNS 블로그 댓글에 대한 감성 모니터링 과제를 수행하려고 합니다.   
자사의 블로그이다보니 \<CNS>, <엘지> 같은 단어들이 많이 보입니다.   

저희 회사 이름이 들어간 만큼 이 토큰들은 subword로 토크나이즈되는 대신 하나의 의미 단위로 분석하고 싶은데요,   
먼저 원래 BERT 단어사전에 이 단어들이 포함되어 있는지 살펴보겠습니다.

In [14]:
print("CNS" in org_vocabs)
print("엘지" in org_vocabs)

False
False


👉 이런, 구글이 공개한 BERT의 단어사전에는 이 토큰들이 포함되어 있지 않습니다.   
👉 그렇다면 지금은 이런 토큰들이 포함된 문장은 어떻게 파싱되고 있는지 확인해보겠습니다.

In [15]:
tokenized = tokenizer.tokenize("안녕하세요 엘지 CNS 임승영 선임 연구원입니다.")
input_ids = tokenizer.convert_tokens_to_ids(tokenized)
print(input_ids)
reversed_token = tokenizer.convert_ids_to_tokens(input_ids)
print(reversed_token)

[9521, 118741, 35506, 24982, 48549, 9562, 12508, 73067, 10731, 9644, 48210, 30858, 9428, 36240, 91785, 14279, 58303, 48345, 119]
['안', '##녕', '##하', '##세', '##요', '엘', '##지', 'CN', '##S', '임', '##승', '##영', '선', '##임', '연구', '##원', '##입', '##니다', '.']


👉 Vocab에 단어가 없다보니 CNS는 CN ##S , 엘지는 엘 ##지 로 찢어져서 토크나이징되고 있습니다. 


이런 현상을 방지하기 위해, 분리되지 않고 분석되었으면 하는 토큰들을 추가해 새로운 단어사전을 만들고   
이를 텍스트 파일로 저장하겠습니다. 

In [16]:
never_split = ["엘지", "CNS"]

## 추가한 never_split 단어를 반영해 새로운 사전을 만들어주기
new_vocabs = org_vocabs.copy()
idx = 1
for tok in never_split:
  if tok not in org_vocabs: # (안전장치 1) 원래 vocab에 없으면
    if "unused" in new_vocabs[idx]: # (안전장치 2) [unused] 토큰 자리이면
      new_vocabs[idx] = tok
      print("{} -> {}".format(org_vocabs[idx], new_vocabs[idx]))
      idx += 1
    else:
      "Cannot Allocate New Token Anymore"
      break

[unused1] -> 엘지
[unused2] -> CNS


👉 new_vocabs에는 never_split으로 정한 토큰들을 포함한 단어 리스트가 저장됩니다. 

새로운 단어를 추가하는 과정에서는 두 가지 안전장치를 넣어주었습니다. 
1. 원래 vocab에 없는 경우에만 추가하기 -> 단어사전에는 중복이 있으면 안 되기 때문입니다.   
2. [unused #] 토큰일 때만 대체하기

In [17]:
## 새 단어사전 저장
new_vocab_file = "/content/new_vocab.txt"

with open(new_vocab_file, "w") as f:
  f.write("\n".join(new_vocabs))

이제 새로 저장한 단어사전을 사용해 new_tokenizer라는 이름으로 다시 한 번 토크나이저를 로딩하겠습니다.

In [18]:
## 새로운 사전을 이용해 로딩

new_tokenizer = bert_tokenization.FullTokenizer(new_vocab_file, do_lower_case)

In [19]:
tokenized = new_tokenizer.tokenize("안녕하세요 엘지 CNS 임승영 선임 연구원입니다.")
print(tokenized)
input_ids = new_tokenizer.convert_tokens_to_ids(tokenized)
print(input_ids)
reversed_token = new_tokenizer.convert_ids_to_tokens(input_ids)
print(reversed_token)

['안', '##녕', '##하', '##세', '##요', '엘지', 'CNS', '임', '##승', '##영', '선', '##임', '연구', '##원', '##입', '##니다', '.']
[9521, 118741, 35506, 24982, 48549, 1, 2, 9644, 48210, 30858, 9428, 36240, 91785, 14279, 58303, 48345, 119]
['안', '##녕', '##하', '##세', '##요', '엘지', 'CNS', '임', '##승', '##영', '선', '##임', '연구', '##원', '##입', '##니다', '.']


👉 토큰을 추가했기 때문에 이번에는 "엘지"와 "CNS"가 쪼개지지 않고 토크나이징되었습니다.   
👉 블로그 댓글에 대한 학습 데이터를 학습하는 과정에서 BERT는 fine-tuning을 통해 새로 추가된 텍스트의 의미를 학습하게 될 것입니다. 

## #5. DAILY MISSION   

<font color="red">MISSION: BERT 토크나이징 & 인덱싱 해보기 </font>

BERT를 사용해 감성분석 과제를 수행하고자 합니다.    
감성분석을 수행하기 위해서는 인풋 문장을 <b>[CLS] 인풋문장 [SEP]</b>의 형태로 만들어야 합니다. 

인풋 문장으로 "BERT 알고 보니 완전 쉽네"라는 문장이 들어왔습니다.   

1. 형태소분석과 BERT 토크나이징을 진행해 위의 문장을 subtokenize하고,    
[CLS] 인풋문장 토큰들 [SEP] 의 형태로 만드세요. 
2. 토크나이즈된 문장을 BERT의 단어사전을 사용해 정수 인덱스로 변환하세요.

<실행 결과는 예시>   
- 형태소 분석 후 -> ['BERT', '알', '고', '보니', '완전', '쉽네']
- Sub-tokenizing 후 -> ['BE', '##RT', '알', '고', '보', '##니', '완', '##전', '쉽', '##네']
- BERT 인풋 형태 변환 -> ['[CLS]', 'BE', '##RT', '알', '고', '보', '##니', '완', '##전', '쉽', '##네', '[SEP]']
- BERT 정수 인덱스 변환 -> [101, 46291, 46935, 9524, 8888, 9356, 25503, 9591, 16617, 9471, 77884, 102]

In [25]:
""" Your Code Here """

sentence = "BERT 알고 보니 완전 쉽네"
 
# basic_tokenizer로 문장 쪼개기
tokenized_sentence = tokenize(sentence)
print(tokenized_sentence)
 
# BPE로 문장 쪼개기
sub_tokens = tokenizer.tokenize(" ".join(tokenized_sentence))
print(sub_tokens)


# [CLS] Subtokens [SEP] 형태로 만들기
sub_tokens = ["[CLS]"] + sub_tokens + ["[SEP]"]
print(sub_tokens)
 
# 정수 인덱스로 변환하기
input_ids = tokenizer.convert_tokens_to_ids(sub_tokens)
print(input_ids)




['BERT', '알', '고', '보니', '완전', '쉽네']
['BE', '##RT', '알', '고', '보', '##니', '완', '##전', '쉽', '##네']
['[CLS]', 'BE', '##RT', '알', '고', '보', '##니', '완', '##전', '쉽', '##네', '[SEP]']
[101, 46291, 46935, 9524, 8888, 9356, 25503, 9591, 16617, 9471, 77884, 102]
